<a href="https://colab.research.google.com/github/AnnaZhuravleva/avtobreja/blob/master/word_lists_expanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
elmo_path = '/content/drive/My Drive/Colab Notebooks/nlp'
project_path = '/content/drive/My Drive/Colab Notebooks/nlp'
sys.path.append(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import urllib
# urllib.request.urlretrieve("https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib", "/content/drive/My Drive/Colab Notebooks/nlp/all.a010.p10.d300.w5.m100.nonorm.slim.joblib")
!pip install git+https://github.com/lopuhin/python-adagram.git
!pip install pymorphy2
import adagram
# from allennlp.commands.elmo import ElmoEmbedder
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import tqdm
import nltk
import json
nltk.download('stopwords')
import numpy as np

!pip install wiki-ru-wordnet
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()

morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')

import gensim
m = '/content/drive/My Drive/Colab Notebooks/nlp/ruscorpora_mystem_cbow_300_2_2015.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
model.init_sims(replace=True)
vm = adagram.VectorModel.load('/content/drive/My Drive/Colab Notebooks/nlp/all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

stops = set(stopwords.words('russian'))

def normalize(text):
    words = [morph.parse(word)[0] for word in tokenize(text) if word not in stops]
    return words

def tokenize(text):
    return token.tokenize(text)

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-vgtsru5s
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-vgtsru5s
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464615 sha256=36503864b75d99aa2836a453125fcbc2043483e5084576c687cabc8c53d96411
  Stored in directory: /tmp/pip-ephem-wheel-cache-wqr8jrbu/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "/content/drive/My Drive/Colab Notebooks/nlp/ruscorpora_mystem_cbow_300_2_2015.bin.gz")

In [0]:
from collections import defaultdict
POS = defaultdict()
POS['INFN'] = 'V'
POS['ADJF'] = 'A'
POS['NOUN'] = 'S'
POS['VERB'] = 'V'

In [0]:
import csv
def lemmas(path):
    words = []
    marks = []
    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for idx, line in enumerate(reader):
            try:
                parsed = normalize(line[0])
                if len(parsed) > 0 and parsed[0].tag.POS in POS.keys() :
                    tmp = str(parsed[0].normal_form) + '_' + POS[str(parsed[0].tag.POS)]
                    words.append(tmp)
                    marks.append(line[1])
            except Exception as e:
                print(e, line)
    words = words[:4000] + words[-4000:]
    marks = [0] * 4000 + [1] * 4000
    return words, marks

**wikiwordnet**

In [0]:
def wwn(lemma):
  wwn = []
  synsets = wikiwordnet.get_synsets(lemma)
  for syns in synsets:
    wwn += [a.lemma() for a in syns.get_words()]
    hyponims = wikiwordnet.get_hyponyms(syns)
    for hyp in hyponims:
      wwn += [a.lemma() for a in hyp.get_words()]
  return set(wwn)

**word2vek**

In [0]:
def w2v(lemma):
  w2v_list = []
  if lemma in model:
      for i in model.most_similar(positive=[lemma], topn=10):
          w2v_list.append(i[0].split('_')[0])
      else:
        pass
  return set(w2v_list)

**adagram**

In [0]:
def adagr(lemma):
  result = []
  try:
      senses = len(vm.word_sense_probs(lemma))
      for i in range(senses):
          result += vm.sense_neighbors(lemma, i)
  except:
      pass
  return set(result)

In [0]:
from copy import deepcopy
def expand_list(path):
    lems, marks = list(lemmas(path))
    print(lems)
    w2v_list, wwn_list, adagr_list, all_lists = deepcopy(lems), deepcopy(lems), deepcopy(lems), {i:j for i,j in zip(lems, marks)}
    i = 0
    for mark, word in zip(marks, lems):
        print(i)
        i += 1
        l1, l2, l3 = w2v(word), wwn(word[:-2]), adagr(word[:-2])
        w2v_list += l1
        wwn_list += l2
        adagr_list += l3
        for a in set(list(l1)+list(l2)+list(l3)):
            all_lists[a] = mark
    print(set(w2v_list), set(wwn_list), set(adagr_list))
    result = set(w2v_list) & set(wwn_list) & set(adagr_list)
    marked_result = {a: all_lists[a] for a in list(result)}
    return marked_result


**составляем списки**

In [0]:
food = '/content/drive/My Drive/Colab Notebooks/nlp/word_lists/semantic_axis_method/food.csv'
service = '/content/drive/My Drive/Colab Notebooks/nlp/word_lists/semantic_axis_method/service.csv'

In [0]:
# import json
#service_l = expand_list(service)
#with open('/content/drive/My Drive/Colab Notebooks/nlp/service.txt', 'w') as outfile:
#    json.dump(service_l, outfile)

In [0]:
#import json
#food_l = expand_list(food)
#with open('/content/drive/My Drive/Colab Notebooks/nlp/food.txt', 'w') as outfile:
#    json.dump(food_l, outfile)

In [0]:
#import json
#with open('/content/drive/My Drive/Colab Notebooks/nlp/service.txt', 'r') as outfile:
#    serv = json.load(outfile)
#import csv
#with open('/content/drive/My Drive/Colab Notebooks/nlp/service.csv', 'w', encoding='utf-8') as outfile:
#      writer = csv.writer(outfile, delimiter='\t')
#      for key in serv:
#          writer.writerow([key, serv[key]])

In [0]:
#import pandas as pd
#pd.read_csv('/content/drive/My Drive/Colab Notebooks/nlp/service.csv', sep='\t').head(20)

In [0]:
def lemmas_keys(path):
    words = []
    with open(path, 'r', encoding='utf-8') as f:
        for idx, line in enumerate(f.readlines()):
            try:
                parsed = normalize(line.strip('\n'))
                if len(parsed) > 0 and parsed[0].tag.POS in POS.keys() :
                    tmp = str(parsed[0].normal_form) + '_' + POS[str(parsed[0].tag.POS)]
                    words.append(tmp)
            except Exception as e:
                print(e, line)
    return words

from copy import deepcopy
def expand_list_keys(path):
    lems = list(lemmas_keys(path))
    raw_lems = [i[:-2] for i in lems]
    w2v_list, wwn_list, adagr_list = deepcopy(raw_lems), deepcopy(raw_lems), deepcopy(raw_lems)
    i = 0
    for word in lems:
        l1, l2, l3 = w2v(word), wwn(word[:-2]), adagr(word[:-2])
        w2v_list += l1
        wwn_list += l2
        adagr_list += [i[0] for i in l3]
    result = w2v_list +  adagr_list
    result = set(result) & set(wwn_list)
    return list(result)

In [18]:
#food_keys = expand_list_keys('/content/drive/My Drive/Colab Notebooks/nlp/word_lists/food_key.txt')
service_keys = expand_list_keys('/content/drive/My Drive/Colab Notebooks/nlp/word_lists/service_key.txt')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


In [19]:
print(food_keys)
print(service_keys)
print(len(food_keys), len(service_keys))

['бутерброд', 'овощ', 'рагу', 'кушанье', 'батон', 'жаркое', 'напиток', 'суп', 'алкоголь', 'десерт', 'кляр', 'повар', 'сыр', 'кетчуп', 'тарелка', 'пицца', 'готовить', 'яство', 'паста', 'состав', 'спирт', 'говядина', 'варить', 'щука', 'кофе', 'гребешок', 'борщ', 'просфора', 'ингредиент', 'зажаривать', 'гуляш', 'тефтели', 'плюшка', 'ветчина', 'пшеница', 'запеканка', 'булочка', 'хлеб', 'баранина', 'морс', 'колбаса', 'выпечка', 'лимонад', 'квас', 'соус', 'молоко', 'печень', 'сок', 'свинина', 'ликёр', 'вино', 'цезарь', 'лосось', 'тунец', 'компот', 'подливка', 'бульон', 'нектар', 'блюдо', 'пунш', 'снедь', 'сосиска', 'обед', 'шампанское', 'краб', 'питие', 'пирог', 'стряпать', 'окунь', 'ужин', 'коктейль', 'песто', 'член', 'кальмар', 'поджаривать', 'жратва', 'пища', 'рыба', 'жарить', 'цена', 'мясо', 'японский', 'пирожок', 'плов', 'яичница', 'порция', 'коньяк', 'сухарь', 'форель', 'салат', 'макароны', 'котлета', 'йогурт', 'вермут', 'винегрет', 'кухня', 'завтрак', 'кока-кола', 'лисичка', 'гренок',

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/nlp/word_lists/service_key_extended.txt','w', encoding='utf-8') as f:
    f.write('\n'.join(service_keys))
with open('/content/drive/My Drive/Colab Notebooks/nlp/word_lists/food_key_extended.txt','w', encoding='utf-8') as f:
    f.write('\n'.join(food_keys))